# Exploratory Factor Analysis

I created a relatively large set of variables during feature building, several of which are probably correlated with one another. For instance, being born in the USA correlates highly with being born on the continent of North America. Now I would like to:

1. Reduce the *dimensionality of the feature space* to help prevent overfitting during model building.
2. Find a representation of the "measured" variables in a lower dimensional space of "unobserved" *latent factors* that span them. Reducing the variables to factors helps with interpretability of models.

The aim is to possibly use the output in building machine learning models that can predict *Nobel Laureates in Physics*.

[Exploratory Factor Analysis](https://en.wikipedia.org/wiki/Exploratory_factor_analysis) (EFA) is a multivariate statistical method that may help with this as it was designed to uncover latent structure in a relatively large set of variables. [Factor Analysis](https://en.wikipedia.org/wiki/Factor_analysis) uses the [correlation matrix](https://en.wikipedia.org/wiki/Correlation_and_dependence#Correlation_matrices) of the variables to examine intercorrelations between the measured variables. It reduces the dimensionality of the matrix by finding groups of variables with high intra-correlation but with low intercorrelation with other groups of variables. A group of these variables is a construct known as a *factor* and in a good factor model the factors have meaning and are appropriately named.

There are several different types of factor models. Since I have a mixture of continuous and categorical (binary) features, the one that seems most appropriate is [Factor Analysis of Mixed Data](https://en.wikipedia.org/wiki/Factor_analysis_of_mixed_data) (FAMD). Roughly speaking, it works as a [Principal Components Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) for quantitative variables and as a [Multiple Correspondence Analysis](https://en.wikipedia.org/wiki/Multiple_correspondence_analysis) (MCA) for categorical variables. It seems to be quite a niche method as there is not too much information about it (especially in English) on the internet. Fortunately there is a nice python library called [prince](https://github.com/MaxHalford/prince) that implements the method along with other factor analysis methods. I will be using the library for my analysis.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from prince import FAMD

%matplotlib inline

## Reading in the Data

First let's read in the training features and the target.

In [ ]:
train_features = pd.read_csv('../data/processed/train-features.csv')
train_features.head()

In [ ]:
target = pd.read_csv('../data/processed/train-target.csv', squeeze=True)
target.head()

## Suitability of Data for Factor Analysis

Any factor analysis always starts with the same question: *Is the data suitable for factor analysis?* There are a few issues to address here. The first are with regards to the *minimum sample size* and *subjects-to-variables* (STV) ratio. There are numerous rules of thumb and various researchers and empirical studies differ in their findings. An excellent review of these are given in chapter 3 of [Best Practices in Exploratory Factor Analysis](https://www.researchgate.net/publication/209835856_Best_Practices_in_Exploratory_Factor_Analysis_Four_Recommendations_for_Getting_the_Most_From_Your_Analysis) and a very good short summary is given in [The Minimum Sample Size in Factor Analysis](https://www.encorewiki.org/display/~nzhao/The+Minimum+Sample+Size+in+Factor+Analysis). To cut a very long story short, basically, my sample size of *N = 540* is deemed sufficient by all researchers and even very good by some. However, my *STV ratio = 540 / 183 = 2.95* is teetering on the brink of unacceptable by many researchers. But it is important to mention that both references give examples of succesful factor analyses for lower values than this.

The last issue concerns *factorability of the correlation matrix* itself. According to Wikiversity's article on [Exploratory Factor Analysis](https://en.wikiversity.org/wiki/Exploratory_factor_analysis), "Factorability is the assumption that there are at least some correlations amongst the variables so that coherent factors can be identified. Basically, there should be some degree of collinearity among the variables but not an extreme degree or singularity among the variables". There are in fact two statistical tests for this: [Bartlett’s test of sphericity](https://en.wikipedia.org/wiki/Bartlett%27s_test) and the [Kaiser–Meyer–Olkin](https://www.statisticshowto.datasciencecentral.com/kaiser-meyer-olkin/) (KMO). However, I'm not going to say too much about these as they are based on the assumption that the data is multivariate normal, which clearly isn't the case here.

The article [Establishing Evidence for Internal Structure Using
Exploratory Factor Analysis](https://www.tandfonline.com/doi/pdf/10.1080/07481756.2017.1336931) suggests that "an intercorrelation matrix is deemed factorable when the majority of the correlation coefficients
computed are in the moderate range wherein r values are between .20 and .80. If a significant
number of variables are producing values below .20 (i.e., items not representing same construct)
or above .80 (i.e., multicollinearity), the researcher should consider eliminating these items
before conducting an EFA (Field, 2013)". OK let's see if this is the case here? Of course taking into consideration the fact that it shouldn't matter if the correlations are positive or negative.

In [ ]:
train_features_numerical = train_features.drop('full_name', axis='columns')
train_features_numerical = train_features_numerical.applymap(
    lambda x: 1 if x == 'yes' else (0 if x == 'no' else x)).applymap(
    lambda x: 1 if x == 'male' else (0 if x == 'female' else x))
correlation = train_features_numerical.corr()
correlation

In [ ]:
print('Percent of correlations in range abs(0.2, 0.8): {0} %'.format(
    100 * round(((abs(correlation) > 0.2) & (abs(correlation) < 0.8)).sum().sum() /
                len(correlation) ** 2, 2))
)     

As you can see, only a small percentage of the values are within this range. This would clearly fail the criteria given above. However, this is not the only viewpoint on this matter. In the article [Exploratory factor analysis: A five-step guide for
novices](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.414.4818&rep=rep1&type=pdf), Tabachnick and Fidell recommended inspecting the correlation matrix (often termed Factorability of R) for correlation coefficients over 0.30... If no correlations go beyond 0.30, then
the researcher should reconsider whether factor analysis is the appropriate statistical method
to utilise." Clearly, there are some correlations above and absolute value of 0.3 in the matrix, so by this criteria, the correlation matrix is factorable. As you can see there are a lot of contrasting recommendations in factor analysis! So for now I will proceed as there are some correlations amongst the variables.

Let me take a little digression for now to explain an important point. Some readers may be wondering why I'm perfectly comfortable using a [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) to measure the correlation between binary-binary or binary-continuous variable pairs? It is for the following reasons: 

1. A Pearson correlation coefficient estimated for two binary variables returns the [phi coefficient](https://en.wikipedia.org/wiki/Phi_coefficient).
2. A [point-biserial_correlation_coefficient](https://en.wikipedia.org/wiki/Point-biserial_correlation_coefficient) is mathematically equivalent to the Pearson correlation coefficient when one variable in the pair is dichotomous and the other continuous.  

## How Many Factors?

It is now time to perform the factor analysis and determine how many factors to retain. Again this is more of an art than a science are there are numerous recommended ways of doing this. Some of the simpler, most straightforward and intuitive ways are:

- *Cumulative variance accounted for by the retained factors*. Here again there are a few recommendations, although most researchers do recommend in the 75-90% range.
- *Scree plot*. A plot of the extracted factors against their eigenvalues in descending order the magnitude. Typically the elbow in the plot is identified where the larger eigenvalues end and the smaller eigenvalues begin. Factors to the left of the elbow are retained and those to the right are dropped. Note that this is quite subjective as sometimes there can be more than one elbow.
- *Kaiser Greater-Than-One-Rule* which says that only those factors with eigenvalues greater than 1 should be retained for interpretation. Again this is arbirtrary, however, an eigenvalue of 1 is the value at which a factor accounts for at least as much variance as any individual variable.

OK let's perform the factor analysis now and use these criteria to decide on the number of factors to retain.

In [ ]:
famd = FAMD(
    n_components=10,
    n_iter=3,
    copy=True,
    random_state=42,
    engine='auto'
)
train_features = train_features.drop('full_name', axis='columns')
famd = famd.fit(train_features)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(8, 6))
ax1 = sns.lineplot(x=range(1, 11), y=famd.eigenvalues_, ax=ax1)
ax1.set_ylim(0, 12e5)
ax1.set_xlabel('Number of factors')
ax1.set_ylabel('Eigenvalues')
ax1.set_title('Scree plot')
ax1.set_xticks(range(0, 11, 2));
ax2 = sns.lineplot(x=range(1, 11), y=np.log10(famd.eigenvalues_), ax=ax2)
ax2.set_ylim((0, 7))
ax2.set_xlabel('Number of factors')
ax2.set_title('Scree plot (log scale)')
ax2.set_ylabel('$log_{10}$(Eigenvalues)')
fig.tight_layout()

In [ ]:
ax = sns.lineplot(x=range(1, 11), y=np.cumsum(famd.explained_inertia_))
ax.figure.set_size_inches((8, 6))
ax.set_xlabel('Number of factors')
ax.set_ylabel('Cumulative variance')
ax.set_title('Cumulative variance accounted for by factors')
ax.set_xlim((0, 10))
ax.set_ylim((0, 1.0))
ax.axhline(y=0.9, linestyle='--', color='r', linewidth=1.0);

The scree plot suggests taking 1 or 2 factors and the Kaiser rule suggests taking any number of factors as the eigenvalues are very large. The cumulative variance accounted by the first 2 factors is about 91% with 89% being accounted for by the first factor. So it seems plausible to retain 2 factors.

However, these are not the only considerations when choosing the number of factors. Very important are the following criteria:

- All factors should be interpretable. In other words, one should be able to coherently name and describe the set of collective variables in an underlying factor.
- There should be several variables that load onto each of the factors. Generally, the more variables per factor, the greater the reliability of the factor. Typically 3 or more variables per factor as a minimum.
- The model should be parsimonius meaning that certain variables should load highly onto a particular factor but load lowly on to other factors. Typically the recommended is loadings with absolute values above 0.3, 0.4 or 0.5 with minimal cross loadings.

With these criteria considered, I find that a factor model with 6 factors seems plausible. To see this, we can examine the tables below. The 0th factor is a *lack of diversity factor* (due to the negative loadings) related to alma mater, workplaces and countries and continents of alma mater, work and residence.

In [ ]:
factor_loadings = famd.column_correlations(train_features)
factor_loadings.loc[factor_loadings[0] < -0.4, 0:5].sort_values(by=0)

Examining the factor scores, unsurprisingly we see that Albert Einstein is the physicist that typifies this diversity factor.

In [ ]:
factor_scores = famd.row_coordinates(train_features)
factor_scores = pd.concat([target, factor_scores], axis='columns')
factor_scores.loc[:, ['full_name', 'physics_laureate', 0]].sort_values(
    by=0, ascending=True)[:10]

The 1st factor is clearly the *North America and Europe distinguishing factor*. It distinguishes associations with North America from those with Europe. These are by far the two continents with the most physicists.

In [ ]:
factor_loadings.loc[factor_loadings[1] > 0.4, 0:5].sort_values(by=1, ascending=False)

The 2nd factor looks very similar to the diversity factor. These two should probably be merged into one factor.

In [ ]:
factor_loadings.loc[factor_loadings[2] > 0.4, 0:5].sort_values(by=2, ascending=False)

The 3rd and 4th factors should probably be merged as the *Great Britain factor*, although they are murked by a few cross-loading variables.

In [ ]:
factor_loadings.loc[factor_loadings[3] > 0.4, 0:5].sort_values(by=3, ascending=False)

In [ ]:
factor_loadings.loc[factor_loadings[4] < -0.4, 0:5].sort_values(by=4, ascending=True)

And the 5th factor is clearly the *Russian factor*. Just look at the list of physicists in the factors score to see how clear this really is! For factors beyond this the loadings are small and the themes are not comprehensible.

In [ ]:
factor_loadings.loc[factor_loadings[5] > 0.4, 0:5].sort_values(by=5, ascending=False)

In [ ]:
factor_scores.join(train_features.born_in_RUS).loc[
    :, ['full_name', 'physics_laureate', 'born_in_RUS', 5]].sort_values(
    by=5, ascending=False)[:10]

Clearly the factor analysis is doing something. However, should I take 2 or 6 factors? It is not really clear. Moreover, there is another issue at play here. The *communality* of a variable is defined as the variance in a given variable explained by all the factors jointly. Clearly the communality of a variable must lie between 0 and 1. [Establishing Evidence for Internal Structure Using
Exploratory Factor Analysis](https://www.tandfonline.com/doi/pdf/10.1080/07481756.2017.1336931) says "Problems exist when communality values are either too high or too low. Communality values equal to or exceeding 1.0 typically indicate a situation where the researcher either extracted too many factors or the initial sample size
from which data were collected was too small. When communality values are close to zero, the associated
variable might be an outlier and distracting from the model. Communality values between .40 and
1.0 indicate that these variables should be retained, as much of the common variance in them can be
explained by the extracted factors (Pett et al., 2003)." 
[Best Practices in Exploratory Factor Analysis](https://www.researchgate.net/publication/209835856_Best_Practices_in_Exploratory_Factor_Analysis_Four_Recommendations_for_Getting_the_Most_From_Your_Analysis) says that "squaring and summing
each factor loading for a variable should equal the extracted communality (within
reasonable rounding error)". OK let's go ahead and compute these communalities.

In [ ]:
for num_comps in range(1, 7):
    famd = FAMD(
        n_components=num_comps,
        n_iter=3,
        copy=True,
        random_state=42,
        engine='sklearn'
    )
    famd = famd.fit(train_features)
    print('\nNumber of components: {0}'.format(num_comps))
    print('Top 35 commulaties : \n{0}'.format(
        (famd.column_correlations(train_features) ** 2).sum(
            axis='columns').sort_values(ascending=False)[:35]))

When the number of components is greater than two, the solutions are spurious since there are variables with communalities greater than one. As a consequence I cannot trust the reliability of those results. However, there is a lot of overlap between the variables in the top communalities. Although the ranking of the variables is different, the majority of the same variables appear, independent of the number of components.

On the other hand, when two factors are extracted the communalities look relatively good. Does the data really live on the latent two-dimensional factor space shown below? If it really does then the data certainly is not linearly separable in this space. What further makes me skeptical is that the factor analysis revealed some higher dimensional latent structure in the data that seems to make sense.

In [ ]:
ax = famd.plot_row_coordinates(
    train_features,
    ax=None,
    figsize=(10, 8),
    x_component=0,
    y_component=1,
    labels=None,
    color_labels=target.physics_laureate,
    ellipse_outline=False,
    ellipse_fill=True,
    show_points=True
)
ax.set_xlabel('Lack of diversity factor ({0} % explained variance)'.format(
    100 * round(famd.explained_inertia_[0], 4)))
ax.set_ylabel('NA / EUR distinguishing factor ({0} % explained variance)'.format(
    100 * round(famd.explained_inertia_[1], 4)))
ax.set_title('Factor plot')
ax.grid(False)
ax.set_xlim(900, 1100)
legend_text = ax.legend().get_texts()
legend_text[0].set_text('non-laureate')
legend_text[1].set_text('laureate')

There are laureates scattered throughout this space and there are outliers. For instance, Albert Einstein is the laureate outlier to far left middle of the plot. Beyond this I do not want to interpret this plot too much as I am very skeptical of the results.

## Discussion

I suspect that the factor analysis results may be invalid due to the STV ratio and / or sample size. However, I also tried increasing the STV ratio by leaving certain variables out of the factor analysis but still found several communalities to be greater than one. I've left out the details for brevity.

Another theory I have is that the features are just too sparse for factor analysis to extract any meaningful information from the correlations in the data. There is some discussion in the context of PCA in [how can one extract meaningful factors from a sparse matrix](https://stats.stackexchange.com/questions/4753/how-can-one-extract-meaningful-factors-from-a-sparse-matrix). If you recall, earlier we saw that only 6% of the values in the correlation matrix had absolute values of correlation coefficients between 0.2 and 0.8. In fact most of the remaining 94% of the values have absolute values of correlation coefficients less than or equal to 0.2.

In [ ]:
print('Percent of correlations less than or equal to abs(0.2): {0} %'.format(
    100 * round((abs(correlation) <= 0.2).sum().sum() / len(correlation) ** 2, 2))
)     

This sparsity was of course induced by the one-hot encoding of variables during feature construction. The point is that most physicists are only associated with a very small fraction of the features. Finding latent structure in such data is difficult. So where does this leave me now?

## Alternatives

There are of course other alternatives for reducing the dimensionality of the data such as:

- [Non-negative matrix factorization](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization) (NMF). This seems attractive as the features matrix consists entirely of non-negative entries.
- [Multidimensional scaling](https://en.wikipedia.org/wiki/Multidimensional_scaling) (MDS). This would have to use a distance "metric" such as the [Gower distance](https://stats.stackexchange.com/questions/15287/hierarchical-clustering-with-mixed-type-data-what-distance-similarity-to-use) since Euclidean distance is just not appropriate for such mixed data types. Unfortunately there is no well established implementation in python, although it may not be too far off in [sklearn](https://github.com/scikit-learn/scikit-learn/issues/5884). There is however a [rudimentary Gower python implementation](https://datascience.stackexchange.com/questions/8681/clustering-for-mixed-numeric-and-nominal-discrete-data), although according to the previous reference, it should be using the Jaccard coefficient for "present" vs "absent" binary variables.

I tried the NMF, but the factors in the feature matrix of reduced dimensionality were uninterpretable to me. Again I have left out the results for the sake of brevity. I did not try the MDS, however, I expect it would suffer from the same interpretability issues as the former. Lack of interpretability is the main drawback with such methods. As interpretability is crucial to this project, these are not viable options.

## Conclusion

The approaches I have taken so far have been fruitless in achieving the two goals I set out to achieve at the outset of this EFA. However there is another approach that can be taken during model building. [Regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)) is a technique that imposes a penalty on more complex models and helps to prevent overfitting. It can lead to models which are simpler, more generalizable and easier to intepret. As Wikipedia says, "Regularization can be used to learn simpler models, induce models to be sparse, introduce group structure into the learning problem, and more." 

During the model building process, I will be looking to take advantage of these properties of regularization. I'm hoping that sparseness in the data, which has proven to be such a problem so far, can in fact be exploited to build a simple, interpretable model that generalizes well.